In [1]:
from preference_generator import create_all_instances, calculate_total_demand, create_noisy_model_student_list, calculate_single_student_demand,calculate_true_bundle_value
from preference_generator_utils import load_obj
import numpy as np

## Generating Student Instances

### Generating Some True Student Lists

In [2]:
# The preferences that were fitted to the Budish and Kessler (2022) data: 
# For 6 popular courses -> mean number of favourites = 3.85
# For 9 popular courses -> mean number of favourites = 2.6
# Maximum budget deviation beta was set to 0.04 as in the Rubenstein 2022 paper
# In our paper, we report results using a supply ratio of 1.1, 1.25 and 1.5


true_student_lists_all_instances, capacities_all_instances, timetables_all_instances = create_all_instances(number_of_instances = 50, number_of_courses= 25, supply_ratio= 1.25, number_of_popular= 9, mean_number_of_favourites= 2.6, maximum_budget_deviation_beta= 0.04, save_results= True, save_folder= "instances")
true_student_lists_all_instances, capacities_all_instances, timetables_all_instances = create_all_instances(number_of_instances = 50, number_of_courses= 25, supply_ratio= 1.25, number_of_popular= 6, mean_number_of_favourites= 3.85, maximum_budget_deviation_beta= 0.04, save_results= True, save_folder= "instances")
true_student_lists_all_instances, capacities_all_instances, timetables_all_instances = create_all_instances(number_of_instances = 50, number_of_courses= 25, supply_ratio= 1.25, number_of_popular= 9, mean_number_of_favourites= 6.1, maximum_budget_deviation_beta= 0.04, additive_preferences= True,save_results= True, save_folder= "instances")

Generating students for run 0
Generating students for run 1
Generating students for run 2
Generating students for run 3
Generating students for run 4
Generating students for run 5
Generating students for run 6
Generating students for run 7
Generating students for run 8
Generating students for run 9
Generating students for run 10
Generating students for run 11
Generating students for run 12
Generating students for run 13
Generating students for run 14
Generating students for run 15
Generating students for run 16
Generating students for run 17
Generating students for run 18
Generating students for run 19
Generating students for run 20
Generating students for run 21
Generating students for run 22
Generating students for run 23
Generating students for run 24
Generating students for run 25
Generating students for run 26
Generating students for run 27
Generating students for run 28
Generating students for run 29
Generating students for run 30
Generating students for run 31
Generating student

## Loading a problem instance

In [3]:
def load_instance(instance_number, supply_ratio = 1.25, number_of_popular = 9, large_grid = True, additive_preferences = False,instance_folder = "instances"):
    """
    A simple function to load a single instance, given its number, the supply ration, and the number of pupular courses
    """

    true_student_list = load_obj(f'{instance_folder}/true_student_lists_sr_{supply_ratio}_popular_{number_of_popular}_lg_{large_grid}_additive_preferences_{additive_preferences}')[instance_number]
    capacities = np.load(f'{instance_folder}/capacities_all_runs_sr_{supply_ratio}_popular_{number_of_popular}_lg_{large_grid}_additive_preferences_{additive_preferences}.npy')[instance_number]
    timetable = load_obj(f'{instance_folder}/timetables_sr_{supply_ratio}_popular_{number_of_popular}_lg_{large_grid}_additive_preferences_{additive_preferences}')[instance_number]

    return true_student_list, capacities, timetable


In [4]:
true_student_list, capacities, timetable = load_instance(instance_number = 2)

In [5]:
# position i of the student list contains the true preferences, and the budget, of the i-th student 
# format: Additive preferences, list of substitute courses, list of complement courses, 3 time parameters are all set to completely off (and designed to model schedule preferences), and finally budget 

print(true_student_list[2])

(array([18.98292144,  8.57514682, 41.8369415 , 16.85689443, 28.37950433,
       95.9478069 , 14.11722218, 19.58535986, 20.38395801, 49.77133774,
       32.46875608, 34.02552264, 44.33089604, 43.40007738, 14.06638234,
       12.56752649, 35.18408194, 41.50088155,  3.68875889, 36.16584376,
        6.33492287, 16.66658431,  1.10659437, 98.20173386,  6.21521183]), [([5, 1], array([ 0. , -0.4])), ([23, 2], array([ 0. , -0.4]))], [([5, 8], array([0. , 0.4]))], 0, 0, [0, 0, 0, 0, 0], 1.0008)


## Creating a "Noisy" version of a student list

In [6]:
number_of_popular = 9 

# these are the noise profiles that were used in our paper and matched as closely as possible the noise profile in the Budish and Kessler 2022 paper. 
if number_of_popular == 9:
    noise_parameter_dictionary = {'noisy_forget_base': 0.5, 'noisy_forget_adjustments': 0.48, 'noisy_base_std': 23, 'noisy_adj_std': 0.2}

elif number_of_popular == 6:
    noise_parameter_dictionary = {'noisy_forget_base': 0.5, 'noisy_forget_adjustments': 0.4825, 'noisy_base_std': 17, 'noisy_adj_std': 0.2}


noisy_student_list = create_noisy_model_student_list(student_list = true_student_lists_all_instances[2], model_type = 'PairwiseAdjustmentsNoisy', 
                                                    seed = 1213, model_param_dictionary = noise_parameter_dictionary)

Noisy all students got called with forget base uniform: 0


## Getting the demands of individual students for a given price vector

In [7]:
price_vector = np.array([0.2 for _ in range(25)])  # A price vector for the individual courses
credit_units = np.array([1 for _ in range(25)])  # the credit units of inidividual courses. For this problem instance, all courses are worth 1 credit unit, and each student wants up to 5 Credit Units

# Demands with respect to their noisy reports in the mechanism
total_demand, individual_demands = calculate_total_demand(prices = np.array([0.2 for _ in range(25)]), student_profiles = noisy_student_list, course_timetable = [[] for  _ in range(5)],
                            credit_units = np.array([1 for _ in range(25)]), model_type = 'PairwiseAdjustmentsNoisy', credit_units_per_student = 5, return_individual_demands = True)
    
# Demands with respect to their true reports (note: these preferences are incompatible with the CM GUI language)
total_demand_true, individual_demands_true = calculate_total_demand(prices = np.array([0.2 for _ in range(25)]), student_profiles = true_student_list, course_timetable = [[] for  _ in range(5)],
                        credit_units = np.array([1 for _ in range(25)]), model_type = 'True', credit_units_per_student = 5, return_individual_demands = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-21


In [8]:
student_number = 7 

demand_noisy = calculate_single_student_demand(prices = np.array([0.2 for _ in range(25)]), 
    student_profile = noisy_student_list[student_number],
    course_timetable = timetable,  # the course timetable affects the feasibility constraints, as students cannot take courses that are scheduled at the same time
    credit_units = [1 for i in range(25)],  # the credit units associated with each course 
    model_type = 'PairwiseAdjustmentsNoisy',  # this is the model type of the GUI reporting language 
    credit_units_per_student = 5,   # by changing this the maximum number of credit units that the student can take changes, and thus so does the maximum number of courses 
    budget = 1.3  # with this parameter you can change the budget of the student. 
    )

demand_true = calculate_single_student_demand(prices = np.array([0.2 for _ in range(25)]), 
    student_profile = true_student_list[student_number],
    course_timetable = timetable,
    credit_units = [1 for i in range(25)],
    model_type = 'True',   # this is the model type of the True student preferences
    credit_units_per_student = 5,    
    budget = 1.3)

## Querying the true value of a student for a bundle

In [9]:
for i in range(len(individual_demands)): 
    value_bundle_noisy_preferences = calculate_true_bundle_value(bundle = individual_demands[i], student_preferences = true_student_list[i], timetable = [[] for  _ in range(5)], make_monotone = True)
    value_bundle_true_preferences = calculate_true_bundle_value(bundle = individual_demands_true[i], student_preferences = true_student_list[i], timetable = [[] for  _ in range(5)], make_monotone = True)

    print(f'For student {i}, value for the optimal bundle {value_bundle_true_preferences} and the one she got under noisy reports: {value_bundle_noisy_preferences}')

For student 0, value for the optimal bundle 411.72519280312423 and the one she got under noisy reports: 236.5388810825664
For student 1, value for the optimal bundle 342.91666879708566 and the one she got under noisy reports: 185.7957492389514
For student 2, value for the optimal bundle 355.16843850217373 and the one she got under noisy reports: 260.7777729223002
For student 3, value for the optimal bundle 322.8647521643826 and the one she got under noisy reports: 272.8847882925842
For student 4, value for the optimal bundle 262.06398045405274 and the one she got under noisy reports: 139.9139578095377
For student 5, value for the optimal bundle 317.1441234097724 and the one she got under noisy reports: 235.8102934334657
For student 6, value for the optimal bundle 318.5573066373473 and the one she got under noisy reports: 176.1232469725648
For student 7, value for the optimal bundle 398.42928565950757 and the one she got under noisy reports: 296.8632911829787
For student 8, value for th